# Project 4 Project
## Recommendations of Movies

There are more than millions of movies made since first introducted but limited time  to see the movies people would like. In order to help consumers save time and money, recommendations are made to make selection easier. To make these recommendation, we will be using a dataset of roughly 10,000 movie entries inorder to predict that a user would rate a given movie higher with those with users with similar reviews on said movie.   

In [1]:
### Read all the dataset and load them in with proper names ###
### Links were not used for this project ###

import pandas as pd
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

# links = pd.read_csv('ml-latest-small/links.csv') 

### Exploration Information
* There are 9742 movies in the dataset
* There are 100836 ratings and 610 users 
* There are 58 users that make up the 3683 tags added
* Genres are separated by | if there are more than one

User to User filtering since there are fewer users than items in the dataset. 

In [2]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [3]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


## Data Preperations

In [4]:
ratings.drop('timestamp', axis = 1, inplace = True)

In [5]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split

reader = Reader()
data = Dataset.load_from_df(ratings,reader)
trainset, testset = train_test_split(data, test_size=0.3, random_state= 69)

In [ ]:
from surprise.prediction_algorithms import knns
from surprise import accuracy

### Baseline Model with KNN Baseline 

In [220]:
sim_cos = {"name": "cosine", "user_based": True}
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)
basic_pred = basic.test(testset)
print(accuracy.rmse(basic_pred))

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9819
0.9818952607403942


#### Baseline Evaulation
The rating are roughly 1 point off which is pretty large considering that the range of ratings are from 1 to 5
This is the first model where we will see if we can imporve on it. We are focused on user to user similarities for this project. 

## Imporved Models 

### SVD algoritm
Find the best parameters for SVD algorithms

In [208]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

# params = {'n_factors': [20, 50, 100],
#          'reg_all': [0.02, 0.05, 0.1],
#           'lr_all': [.001, .002, .003, .004, .005],
#          'random_state':[69]
#          }
# g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
# g_s_svd.fit(data)
# g_s_svd.best_params

#  {'rmse': {'n_factors': 100,
#   'reg_all': 0.05,
#   'lr_all': 0.005,
#   'random_state': 69},
#  'mae': {'n_factors': 100,
#   'reg_all': 0.05,
#   'lr_all': 0.005,
#   'random_state': 69}}

In [ ]:
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline

In [211]:
best_svd = SVD(n_factors= 20, reg_all = 0.1)
best_svd.fit(trainset)
svd_pred = best_svd.test(testset)
print(accuracy.rmse(svd_pred))

RMSE: 0.8791
0.8791224608358874


### Memory Based Methods 

#### KnnBasic

In [221]:
sim_pearson = {"name": "pearson", "user_based": True}

basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(trainset)
predictions = basic_pearson.test(testset)
print(accuracy.rmse(predictions))   

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9849
0.9849105382524579


#### Knn With Means

In [213]:
sim_pearson = {"name": "pearson", "user_based": True}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9088
0.9087933637961305


#### Knn Baseline 

In [214]:
sim_pearson = {"name": "pearson", "user_based": True}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson)
knn_baseline.fit(trainset)
predictions = knn_baseline.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8893
0.889324777085977


### KnnBaseline vs SVD

In [215]:
clf = knns.KNNBaseline(sim_options=sim_pearson)
params = {'k':[10, 20, 30, 40, 50],
          'min_k': [1, 2, 3, 4],
          'random_state':[69]
         }
g_s_baseline = GridSearchCV(KNNBaseline,param_grid=params,n_jobs=-1)
g_s_baseline.fit(data)

In [216]:
g_s_baseline.best_params

{'rmse': {'k': 30, 'min_k': 4, 'random_state': 69},
 'mae': {'k': 30, 'min_k': 4, 'random_state': 69}}

In [218]:
sim_pearson = {"name": "pearson", "user_based": True}
knn_baseline = knns.KNNBaseline(k = 30, min_k = 4,sim_options=sim_pearson)
knn_baseline.fit(trainset)
predictions = knn_baseline.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8776
0.8776019532148854


In [203]:
def user_top_5(user_id):
    if(user_id not in ratings['userId'].unique()):
        return "Invalid User"
    seen_movies = ratings['movieId'].loc[ratings['userId'] == user_id].values
    new_movies = [movie for movie in movies.movieId.values if movie not in seen_movies]
    predicted_ratings = []
    for movie in new_movies:
        predicted_ratings.append((movie, best_svd.predict(user_id, movie)[3]))
    list_of_movies = sorted(predicted_ratings, key= lambda x: x[1], reverse= True)
    movie_recommendations = movies['title'].loc[movies['movieId']
    return list_of_movies

def past_movies(user_id):
    user_movies = ratings.loc[ratings['userId'] == user_id]
    ids = [ID for ID in user_movies['movieId'].values]
    past_movies = movies[['title', 'genres']].loc[movies['movieId'].isin(ids)]
    return past_movies

def count_genres(movie_df):
    genres

SyntaxError: invalid syntax (<ipython-input-203-145a2eb37937>, line 11)

In [ ]:
user_top_5(1)

In [196]:
movie_df = past_movies(1)
list_genres = movie_df.genres.values

In [194]:
genres = {}
for movie in list_genres: 
    separated_genre = movie.split(sep= '|')
    for genre in separated_genre:
        if genres.get(genre) != None:
            genres[genre] += 1
        else:
            genres[genre] = 1
sorted(genres.items(), key= lambda x : x[1], reverse= True)

[('Action', 90),
 ('Adventure', 85),
 ('Comedy', 83),
 ('Drama', 68),
 ('Thriller', 55),
 ('Fantasy', 47),
 ('Crime', 45),
 ('Children', 42),
 ('Sci-Fi', 40),
 ('Animation', 29),
 ('Romance', 26),
 ('War', 22),
 ('Musical', 22),
 ('Mystery', 18),
 ('Horror', 17),
 ('Western', 7),
 ('Film-Noir', 1)]